In [2]:
import tensorflow as tf

# from gensim.test.utils import common_texts,get_tmpfile
from gensim.models import word2vec
from gensim import corpora


from nltk.corpus import reuters,stopwords,brown
from nltk.tokenize import sent_tokenize,word_tokenize,RegexpTokenizer
# from nltk import Text

import numpy as np
import pandas as pd
import string
import re
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder

# from pprint import pprint

unable to import 'smart_open.gcs', disabling that module


In [3]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [4]:
# id_ls = pd.DataFrame(reuters.fileids())

corp_select = reuters

id_ls = pd.DataFrame(corp_select.fileids())
categ_ls = pd.DataFrame(corp_select.fileids())

text = corp_select.raw(fileids=id_ls[0])
# text = brown.raw(categories='humor')
sen_ls = sent_tokenize(text)[:1000]

In [5]:
trans=str.maketrans({key: None for key in string.punctuation}) #建立转换关系

In [6]:
word_sen_ls = []
word_ls = []
reg_tokenizer_b = RegexpTokenizer(r'\s+',gaps=True) ## 按照空格划分
reg_tokenizer_w = RegexpTokenizer(r'\w+') ## 按照单词

for sen in sen_ls[:1000]:
    # word_ls.append(word_tokenize(sen))

    word_ls_tmp = reg_tokenizer_b.tokenize(sen)
    word_ls_tmp = list(filter(lambda x:x not in stopwords.words('english'),word_ls_tmp))
    word_ls_tmp = list(filter(lambda x:x not in [*map(lambda x:x.upper(),stopwords.words('english'))],word_ls_tmp))

    word_ls_tmp = list(map(lambda x:re.sub("[0-9]+","",x),word_ls_tmp))
    word_ls_tmp = list(map(lambda x:x.translate(trans),word_ls_tmp))
    word_sen_ls.append(word_ls_tmp)
    word_ls = word_ls + word_ls_tmp


vob = np.unique(word_ls)


In [7]:
len(word_sen_ls)

1000

In [8]:
vob_size = len(vob)
len(word_ls),len(vob)

(18382, 4742)

In [55]:
def gen_x_y(sentence):

    window = 4
    n = len(sentence)
    # print(n)
    # if n<=window:
    #     pass

    x_input = []
    y_input = []

    for k in range(n-4): #### 遍历句子长度 ####
        for w in range(window): #### 对窗口内所有单词组合全部拿出来 ####
            x_tmp,y_tmp = sentence[k+w],sentence[k+window]

            x_input.append(x_tmp)
            y_input.append(y_tmp)

    for k in range(n-1,4,-1): #### 遍历句子长度 ####
        # print(k)
        for w in range(window): #### 对窗口内所有单词组合全部拿出来 ####
            x_tmp,y_tmp = sentence[k-w],sentence[k-window]

            x_input.append(x_tmp)
            y_input.append(y_tmp)

        # print(x_tmp,y_tmp)

    x_input = pd.DataFrame(x_input)
    y_input = pd.DataFrame(y_input)

    return x_input,y_input

In [56]:
X = pd.DataFrame()
y = pd.DataFrame()

for sen in word_sen_ls:

    if len(sen)<=5:
        continue

    X_tmp,y_tmp = gen_x_y(sen)

    X = pd.concat([X,X_tmp])
    y = pd.concat([y,y_tmp])

In [57]:
X.shape,y.shape

((111376, 1), (111376, 1))

In [58]:
X.head()

,0
0,ASIAN
1,EXPORTERS
2,FEAR
3,DAMAGE
4,EXPORTERS


In [59]:
# ohe = OneHotEncoder()
ohe = OrdinalEncoder()
ohe.fit(vob.reshape(-1,1))

OrdinalEncoder(categories='auto', dtype=<class 'numpy.float64'>)

In [60]:
X_trans = X.apply(lambda x:np.squeeze(ohe.transform(x.reshape(-1,1)),axis=1))
y_trans = y.apply(lambda x:np.squeeze(ohe.transform(y[0].reshape(-1,1))))

In [61]:
X_trans = X_trans.astype(int)
y_trans = y_trans.astype(int)

In [62]:
X_trans.head()

,0
0,30
1,484
2,537
3,387
4,484


In [64]:
y_trans.head()

,0
0,1611
1,1611
2,1611
3,1611
4,1274


In [65]:
#### RNN CELL ####
rnn_mod = 0
num_unit = 32

if rnn_mod==0:
    cell = tf.nn.rnn_cell.BasicRNNCell(num_units=num_unit)
if rnn_mod==1:
    cell = tf.nn.rnn_cell.RNNCell(num_units=num_unit)
if rnn_mod==2:
    cell = tf.nn.rnn_cell.LSTMCell(num_units=num_unit)
if rnn_mod==3:
    cell = tf.nn.rnn_cell.GRUCell(num_units=num_unit)
    

In [66]:
#### word embedding ###
embed_size = 100

input_x = tf.placeholder(tf.int32,[None])
input_y = tf.placeholder(tf.int32,[None,1])

word_embeddings = tf.Variable(tf.truncated_normal([vob_size,embed_size],stddev=1,mean=0.1))
embedding_word_ids = tf.nn.embedding_lookup(word_embeddings,input_x)

weights = tf.Variable(tf.truncated_normal([vob_size,embed_size],mean=1,stddev=1))
bias = tf.Variable(tf.truncated_normal([vob_size]))

In [67]:
loss = tf.reduce_sum(
    tf.nn.sampled_softmax_loss(weights=weights,
                                biases = bias,
                                labels = input_y,
                                inputs = embedding_word_ids,
                                num_sampled = 20,
                                num_classes = vob_size
                                ))

optimizer = tf.train.AdamOptimizer(learning_rate=0.01).minimize(loss)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [69]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for i in range(400):
        _,losses,wgs = sess.run([optimizer,loss,weights],feed_dict={input_x:np.squeeze(X_trans.values),
                                                  input_y:y_trans})

        if i%10==0:
            print(i,losses)

0 1479593.2
10 1155886.2
20 1024167.06
30 790968.2
40 569916.25
50 602854.2
60 505109.38
70 401911.94
80 340491.25
90 310117.9
100 340358.7
110 341198.72
120 263772.8
130 292137.4
140 190056.27
150 322103.06
160 278547.88
170 343993.56
180 253769.53
190 214409.47
200 404969.75
210 147878.48
220 200520.56
230 246971.25
240 243187.44
250 212383.38
260 98627.6
270 356667.38
280 379110.3
290 204087.4
300 258962.19
310 113330.484
320 128491.49
330 171816.05
340 104803.125
350 220149.16
360 189721.1
370 139121.47
380 108038.67
390 69707.734


In [70]:
wgs.shape

(4742, 100)

array([[4431.]])

In [88]:
vector1 = wgs[int(ohe.transform([['money']])[0,0])]

In [89]:
vector2 = np.dot(wgs,vector1)
order2 = vector2.argsort()[::-1]

In [90]:
ohe.inverse_transform(order2[:10,np.newaxis])

array([['also'],
       ['would'],
       ['one'],
       ['pay'],
       ['despite'],
       ['buy'],
       ['month'],
       ['Group'],
       ['including'],
       ['government']], dtype='<U19')

In [47]:
for k in range(10,4,-1):
    print(k)

10
9
8
7
6
5
